In [ ]:
# pip install httpx websockets

In [27]:
#!/usr/bin/env python3
import yaml
import json
import asyncio
import websockets
import time

# Install nest_asyncio if you haven't already:
# !pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()  # This allows asyncio.run() inside Jupyter


In [40]:
# Load the YAML flow file
yaml_file_path = "./execution_flows/simple-ai-flow.yaml"  # Update this to your file location
print(f"Loading flow from {yaml_file_path}...")

with open(yaml_file_path, 'r') as file:
    yaml_data = yaml.safe_load(file)

flow_id = yaml_data.get("flow_id", "flow-" + yaml_file_path.split("/")[-1].split(".")[0])
print(f"Flow ID: {flow_id}")


Loading flow from ./execution_flows/simple-ai-flow.yaml...
Flow ID: simple_ai_flow


In [42]:



async def execute_and_stream():
    websocket_url = f"ws://localhost:8000/ws/execute/{flow_id}"
    print(f"Connecting to WebSocket at {websocket_url}")
    
    try:
        async with websockets.connect(websocket_url) as websocket:
            # Receive ready message
            ready_msg = await websocket.recv()
            print(f"Server: {ready_msg}")
            
            # Send flow definition
            flow_definition_str = json.dumps(yaml_data.get("flow_definition"))
            await websocket.send(flow_definition_str)
            print("Sent flow definition")
            
            # Receive acknowledgment
            ack1 = await websocket.recv()
            print(f"Server: {ack1}")
            
            # Send initial inputs
            initial_inputs_str = json.dumps(yaml_data.get("initial_inputs", {}))
            await websocket.send(initial_inputs_str)
            print("Sent initial inputs")
            
            # Receive acknowledgment
            ack2 = await websocket.recv()
            print(f"Server: {ack2}")
            
            # Send config (null in this case)
            await websocket.send("null")
            print("Sent null config")
            
            # Receive final acknowledgment
            ack3 = await websocket.recv()
            print(f"Server: {ack3}")
            
            print("Flow execution starting. Receiving events...")
            
            # Now receive streaming events
            all_output = ""
            while True:
                try:
                    message = await websocket.recv()
                    data = json.loads(message)
                    event_type = data.get("type", "unknown")
                    
                    if event_type == "llm_chunk":
                        chunk = data.get("data", {}).get("content", "")
                        all_output += chunk
                        print(chunk, end="", flush=True)
                    elif event_type == "flow_completed":
                        print("\n\nFlow completed!")
                        break
                    elif event_type == "element_error" or event_type == "flow_error":
                        error = data.get("data", {}).get("error", "Unknown error")
                        print(f"\nError: {error}")
                        break
                except websockets.exceptions.ConnectionClosed:
                    print("\nWebSocket connection closed")
                    break
            
            print("\n--- Complete response ---\n")
            print(all_output)
    
    except Exception as e:
        print(f"Error: {str(e)}")

# In Jupyter, don't use asyncio.run() - use one of these approaches:

# OPTION 1: If using Jupyter with IPython 7+, you can await directly:
await execute_and_stream()

# OPTION 2: If the above doesn't work, use nest_asyncio (already applied at the top):
# asyncio.run(execute_and_stream())

Connecting to WebSocket at ws://localhost:8000/ws/execute/simple_ai_flow
Server: {"status": "ready", "message": "Send flow_definition as JSON"}
Sent flow definition
Server: {"status": "received_flow", "message": "Send initial_inputs as JSON"}
Sent initial inputs
Server: {"status": "received_inputs", "message": "Send config as JSON or 'null'"}
Sent null config
Server: {"status": "starting", "message": "Starting flow execution"}
Flow execution starting. Receiving events...


Okay, let's see. The user is asking for the capital of France and its significance. Since they mentioned being a geography enthusiast, they probably want more than just the basic answer. First, I need to confirm that Paris is indeed the capital. I remember that Paris has been the capital for a long time, but maybe I should check if there were any historical changes. For significance, I should think about its political, cultural, and economic roles. Politically, it's the seat of government, home to institutions like t